<a href="https://colab.research.google.com/github/jared-ni/6.8610-project/blob/main/claude_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
!pip install googletrans==4.0.0-rc1
!pip install deep-translator
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-no

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import userdata
claude_token = userdata.get('claude_token')

In [4]:
import csv
from datasets import load_dataset
import pandas as pd
import spacy
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm

In [5]:
# Flags for which LLMs to use
USE_LLAMA = True
USE_MISTRAL = False
USE_FALCON = False

# Define max_length multiplier for LLM prompts
MAX_LENGTH_MULTIPLIER = 2

# Define k hyperparameter
K_HYPERPARAMETER = 3

In [6]:
# Load datasets into pandas DataFrames
def load_law_dataset():
    ds = load_dataset("casehold/casehold", "all")
    train_df = pd.DataFrame(ds['train'])
    test_df = pd.DataFrame(ds['test'])
    validation_df = pd.DataFrame(ds['validation'])
    law_dataset = pd.concat([train_df, test_df, validation_df], ignore_index=True)['citing_prompt']
    return law_dataset

def load_medical_dataset():
    ds = load_dataset("zhengyun21/PMC-Patients")
    train_df = pd.DataFrame(ds['train'])
    medical_dataset = train_df['patient']
    return medical_dataset

# Combine datasets
def load_all_datasets():
    law_dataset = load_law_dataset()
    medical_dataset = load_medical_dataset()
    return [law_dataset, medical_dataset]

In [7]:
# Load SpaCy model
def load_spacy_model(model_path='en_core_sci_sm'):
    return spacy.load(model_path)

# Extract entities from text
def extract_entities(nlp, text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

# Translate entities to a target language
def translate_entities(entities, target_lang):
    translations = [GoogleTranslator(source='auto', target=target_lang).translate(entity) for entity in entities]
    return translations

In [8]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.7 MB/s eta 0:00:00


In [9]:
from itertools import combinations
import jieba
import sacrebleu

# Calculate JTC score
def calculate_JTC(translations, text, entities):
    jtc_score = 0
    n = len(entities)
    entity_counts = 0

    for entity, translated_entity in entities.items():
        # Count occurrences of the entity in the original text
        c = text.count(entity)
        entity_counts += c
        if c == 0 or not translated_entity:
            continue  # Skip entities that are not present or have no translation
        jargon_inconsistency = K_HYPERPARAMETER * c

        for translation in translations:
            # Count occurrences of the translated entity in the translation
            t = translation.count(translated_entity)
            # Calculate penalty for mismatched occurrences
            jargon_inconsistency -= t

        # Update the JTC score
        jtc_score += jargon_inconsistency

    # Normalize and invert the score
    normalized_score = jtc_score / max(K_HYPERPARAMETER * entity_counts, 1)
    return 1 - normalized_score

# calculates the Jaccard similarity between the k translations
def calculate_jaccard(translations, target_lang):
    jaccard_scores = []
    if target_lang == "Simplified Chinese":
        translations = [set(jieba.lcut(translation)) for translation in translations]
    else:
        translations = [set(translation.split()) for translation in translations]
    # print("translations:", translations)
    for t1, t2 in combinations(translations, 2):
        # Calculate Jaccard similarity for the pair
        intersection = len(t1.intersection(t2))
        union = len(t1.union(t2))
        jaccard_score = intersection / union if union > 0 else 0
        jaccard_scores.append(jaccard_score)
    # print("Jaccard Scores:", jaccard_scores)
    # Return the average Jaccard similarity
    return sum(jaccard_scores) / len(jaccard_scores) if jaccard_scores else 0

# chrF++ ⇒ google translate serves as a reference ground truth
def calculate_chrf(ground_truth_translation, translations, n_value=6):
    ground_truth_translations = [ground_truth_translation] * len(translations)
    # sacrebleu.corpus_chrf calculates chrF++ directly
    chrf = sacrebleu.corpus_chrf(
        translations,  # List of translated texts
        ground_truth_translations,  # List of reference texts
        beta=2  # Default beta for F-score weighting
    )
    return chrf.score

In [10]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have ht

In [11]:
# import anthropic

# api_key = userdata.get('claude_token')
# client = anthropic.Anthropic(api_key=api_key)

# message = client.messages.create(
#     # model="claude-3-5-sonnet-20241022",
#     model="claude-3-haiku-20240307",
#     max_tokens=512,
#     messages=[
#         {"role": "user", "content": "Please translate this to Chinese: The patient was diagnosed with diabetes and is very sick"}
#     ]
# )
# print(message.content)

In [17]:
import anthropic
import time
import random

# Helper function to interact with Claude with retries and exponential backoff
def invoke_claude(prompt, model="claude-3-haiku-20240307", max_tokens=512, max_retries=5):
    api_key = userdata.get('claude_token')
    client = anthropic.Anthropic(api_key=api_key)
    retries = 0
    backoff = 1  # Start with 1-second backoff

    while retries < max_retries:
        try:
            # Make the API request
            response = client.messages.create(
                model=model,
                max_tokens=max_tokens,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.content[0].text  # Return the response if successful
        except Exception as e:
            print(f"Error occurred: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying (attempt {retries}/{max_retries})...")
                time.sleep(backoff)
                backoff *= 2  # Exponential backoff
                backoff += random.uniform(0, 1)  # Add jitter
            else:
                print(f"Max retries reached for prompt: {prompt}. Skipping.")
                return None  # Skip after max retries


In [13]:
from tqdm import tqdm

In [18]:
# Main function for the experiment
def run_pipeline_claude(target_lang, dataset_index=0, results_file_in_drive="results.csv"):
    datasets = load_all_datasets()
    lang_abbrs = {"Simplified Chinese": "zh-CN", "French": "fr"}

    # Path to the results file in Google Drive
    results_path = f'/content/drive/My Drive/{results_file_in_drive}'

    # Initialize the CSV file with headers
    with open(results_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Text Index", "Regular_JTC_Score", "LEAP_JTC_Score",
                         "Regular_Jaccard_Score", "LEAP_Jaccard_Score",
                         "Regular_chrF", "LEAP_chrF"])

    dataset = datasets[dataset_index]
    dataset_name = "law" if dataset_index == 0 else "medical"
    print(f"Processing dataset: {dataset_name}")

    for i, text in enumerate(tqdm(dataset[:500], desc=f"Processing {dataset_name} dataset")):

        if target_lang == "Simplified Chinese" and dataset_index == 0 and i <= 440:
            continue # this is to pick up where we left off from the initial api server crash

        text = " ".join(text.split()[:50])  # Truncate to the first 50 words

        named_entities = list(set(extract_entities(load_spacy_model(), text)))
        named_entities_translations = translate_entities(named_entities, lang_abbrs[target_lang])
        named_entity_mapping = {e: t for e, t in zip(named_entities, named_entities_translations)}

        # Regular text translation
        regular_translations = []
        for k in range(3):  # Adjust K_HYPERPARAMETER as needed
            prompt = f"Please return only the translation and nothing else. Translate the following text to {target_lang}: {text}"
            response = invoke_claude(prompt)
            if response is None:
                print(f"Skipping text index {i + 1} due to failed regular translation.")
                break  # Skip this text if any translation fails
            regular_translations.append(response)

        # If we skipped due to a failure, continue to the next text
        if len(regular_translations) < 3:
            continue

        # LEAP text translation
        leap_translations = []
        for k in range(3):  # Adjust K_HYPERPARAMETER as needed
            prompt = f"Please return only the translation and nothing else. Translate the following text into {target_lang} using these mappings {str(named_entities_translations)}: {text}"
            response = invoke_claude(prompt)
            if response is None:
                print(f"Skipping text index {i + 1} due to failed LEAP translation.")
                break  # Skip this text if any translation fails
            leap_translations.append(response)

        # If we skipped due to a failure, continue to the next text
        if len(leap_translations) < 3:
            continue

        # Calculate JTC scores
        regular_jtc_score = calculate_JTC(regular_translations, text, named_entity_mapping)
        leap_jtc_score = calculate_JTC(leap_translations, text, named_entity_mapping)

        # Calculate Jaccard Similarities
        regular_jaccard_score = calculate_jaccard(regular_translations, target_lang)
        leap_jaccard_score = calculate_jaccard(leap_translations, target_lang)

        # chrF++ Metric
        ground_truth_translation = GoogleTranslator(source='auto', target=lang_abbrs[target_lang]).translate(text)
        regular_chrf = calculate_chrf(ground_truth_translation, regular_translations)
        leap_chrf = calculate_chrf(ground_truth_translation, leap_translations)

        # Append results to the file in Google Drive
        with open(results_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            row = [i + 1, regular_jtc_score, leap_jtc_score,
                   regular_jaccard_score, leap_jaccard_score,
                   regular_chrf, leap_chrf]
            writer.writerow(row)
            print(f"Appended result to Google Drive file: {row}")

In [19]:
# Run the pipeline for different target languages and datasets
target_langs = ["Simplified Chinese", "French"]
dataset_indices = [0, 1]  # 0: Law, 1: Medical

# Outer loop for target languages
for target_lang in tqdm(target_langs, desc="Target Languages"):
    # Inner loop for dataset indices
    for dataset_index in tqdm(dataset_indices, desc=f"Datasets for {target_lang}", leave=False):
        results_file = f"full_claude_{target_lang}_{dataset_index}_results.csv"
        run_pipeline_claude(target_lang=target_lang,
                            dataset_index=dataset_index,
                            results_file_in_drive=results_file)

Datasets for Simplified Chinese:   0%|          | 0/2 [00:00<?, ?it/s]

Processing dataset: law




Processing law dataset:   0%|          | 0/500 [00:00<?, ?it/s]

Processing law dataset:  88%|████████▊ | 442/500 [00:11<00:01, 38.67it/s]

Appended result to Google Drive file: [442, 0.5777777777777777, 0.9333333333333333, 0.27067639673571875, 0.9407114624505929, 7.5, 5.9171597633136095]
Appended result to Google Drive file: [443, 0.08333333333333337, 1.5833333333333335, 0.6666666666666666, 0.43256172839506174, 75.0, 3.947368421052632]




Processing law dataset:  88%|████████▊ | 442/500 [00:27<00:01, 38.67it/s]

Processing law dataset:  89%|████████▉ | 444/500 [00:36<00:05,  9.41it/s]

...



Processing medical dataset: 100%|█████████▉| 499/500 [1:30:40<00:10, 10.97s/it]

Appended result to Google Drive file: [499, 0.47916666666666663, 0.9375, 0.4401066651616909, 0.11433733384952897, 2.2522522522522523, 1.506024096385542]




Processing medical dataset: 100%|██████████| 500/500 [1:30:50<00:00, 10.90s/it]

Appended result to Google Drive file: [500, 0.8444444444444444, 0.8888888888888888, 0.960392156862745, 0.9215686274509803, 1.8083182640144666, 1.8148820326678767]




Target Languages: 100%|██████████| 2/2 [5:52:09<00:00, 10564.85s/it]
